In [36]:
import os

In [37]:
%pwd

'/Users/siddharth/Documents/EndtoEndProjects/TextSummarizerProject'

In [38]:
os.chdir("/Users/siddharth/Documents/EndtoEndProjects/TextSummarizerProject")

In [39]:
%pwd

'/Users/siddharth/Documents/EndtoEndProjects/TextSummarizerProject'

In [40]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [41]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [42]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [43]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [44]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Disable MPS and force CPU
        os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
        torch.backends.mps.is_available = lambda: False
        device = "cpu"

        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        # Load tokenized dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Ultrafast TrainingArguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.eval_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            fp16=False,                  
            dataloader_num_workers=0,   
            save_total_limit=1,         
            report_to=[],               
        )

        trainer = Trainer(
            model=model,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        # Save model and tokenizer
        model.save_pretrained(os.path.join(self.config.root_dir, "t5-small"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [45]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-07-19 23:23:09,832: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-19 23:23:09,834: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-19 23:23:09,835: INFO: common: created directory at: artifacts]
[2025-07-19 23:23:09,835: INFO: common: created directory at: artifacts/model_trainer]


  1%|          | 5/819 [00:02<06:53,  1.97it/s]

                                          


                                                 


  1%|          | 5/819 [00:02<06:53,  1.97it/s]

{'loss': 2.9257, 'grad_norm': 8.752639770507812, 'learning_rate': 5e-05, 'epoch': 0.01}


  1%|          | 10/819 [00:03<04:46,  2.83it/s]

                                          


                                                 


  1%|          | 10/819 [00:03<04:46,  2.83it/s]

{'loss': 3.1741, 'grad_norm': 15.644474983215332, 'learning_rate': 4.9692874692874694e-05, 'epoch': 0.01}


...

{'eval_loss': 1.9525151252746582, 'eval_runtime': 35.2959, 'eval_samples_per_second': 23.175, 'eval_steps_per_second': 23.175, 'epoch': 0.99}


100%|█████████▉| 815/819 [50:54<00:11,  2.94s/it]

                                          


                                                 


100%|█████████▉| 815/819 [50:54<00:11,  2.94s/it]

{'loss': 2.1544, 'grad_norm': 5.987986087799072, 'learning_rate': 2.457002457002457e-07, 'epoch': 1.0}


100%|██████████| 819/819 [50:55<00:00,  1.10it/s]

                                          


                                                 


100%|██████████| 819/819 [50:55<00:00,  3.73s/it]


{'train_runtime': 3055.8511, 'train_samples_per_second': 0.268, 'train_steps_per_second': 0.268, 'train_loss': 2.2896759018065436, 'epoch': 1.0}
